https://docs.python.org/3.4/library/multiprocessing.html

In [1]:
import sys
print(sys.version)
import pandas
print('pandas',pandas.__version__)
import numpy
print('numpy',numpy.__version__)
from multiprocessing import Process
from multiprocessing import Pool
import os
import time

3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
pandas 0.23.4
numpy 1.13.3


# introduction to processes

Your computer automatically assigns a "process ID" (pid) to each process

https://en.wikipedia.org/wiki/Process_identifier

In [2]:
print('process ID =',os.getpid())

process ID = 131


In [3]:
# what launched this program?
print('parent process =', os.getppid())

parent process = 6


before using multiprocessing, inspect the processes of a normal python execution

In [5]:
def proc_info(some_str):
    print('  Now in "proc_info"')
    print('  arg =',some_str) # string that was passed in
#    print('  module name:', __name__)
    print('  process id:', os.getpid())

def myfunc(a_name):
    """ this function calls another function """
    print('Now in "myfunc"')
    print('process ID =',os.getpid())
    proc_info(a_name)
    print('end of "myfunc"')

In [6]:
myfunc('mary')

Now in "myfunc"
process ID = 131
  Now in "proc_info"
  arg = mary
  process id: 131
end of "myfunc"


<BR>
<BR>
<BR>

--> functions run in the current process
<BR>
<BR>


# launch a separate (serial) process<BR> using multiprocessing module

In [7]:
if __name__ == '__main__':
    print('process ID =',os.getpid())
    proc_info('cool stuff') # display info about this process
    
    print('\ncreate process')
    p = Process(target=myfunc, args=('bob',)) # from main, call a function
    
    print('\nstart process')
    p.start()

process ID = 131
  Now in "proc_info"
  arg = cool stuff
  process id: 131

create process

start process
Now in "myfunc"
  Now in "proc_info"
process ID = 255
  arg = bob
  process id: 255
end of "myfunc"


# multiprocessing for concurrent operations
Now that we've explored having multiple processes, use the processes concurrently

https://docs.python.org/3.4/library/multiprocessing.html#using-a-pool-of-workers

In [8]:
from multiprocessing import Pool

## _example_: map function to list

In [9]:
def sq_fun(x):
    """
    square the input
    """
    return x*x

In [10]:
%timeit [sq_fun(x) for x in [1, 2, 3]]

3.81 µs ± 2.19 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
%%time

if __name__ == '__main__': # only the parent thread runs the pool
    
    with Pool(5) as p: # use of 'with' means we can avoid use of close
        print(p.map(sq_fun, [1, 2, 3]))

[1, 4, 9]
CPU times: user 74.5 ms, sys: 267 ms, total: 341 ms
Wall time: 508 ms


for "microseconds" (us, 1E-6) versus "milliseconds" (ms, 1E-3), see https://en.wikipedia.org/wiki/Metric_prefix#List_of_SI_prefixes

1 ms = 1000 us

--> Why did it take so much longer?

## _example_: dataframe operation

Objective: instead of applying a function to a dataframe row-by-row, 
split the dataframe into chunks and apply the function concurrently using multiple processes

First, create a random dataframe

https://stackoverflow.com/questions/32752292/how-to-create-a-data-frame-of-random-integers-with-pandas

In [12]:
df = pandas.DataFrame(
    numpy.random.randint(0,100,size=(100, 4)), 
    columns=list('ABCD')
    )

In [14]:
print('df shape:',df.shape)
df.head()

df shape: (100, 4)


,A,B,C,D
0,49,3,96,64
1,13,39,99,67
2,61,45,76,68
3,88,93,75,74
4,8,26,49,54


http://www.racketracer.com/2016/07/06/pandas-in-parallel/

In [15]:
num_partitions = 7 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

In [16]:
def parallelize_dataframe(df, func,num_cores):
    """
    Apply a function to chunks of a dataframe concurrently
    """
    df_split = numpy.array_split(df, num_partitions)
    with Pool(num_cores) as pool:
        df = pandas.concat(pool.map(func, df_split))
    return df

In [17]:
def multiply_columns(data):
    data['K'] = data['A'].apply(lambda x: x*2)
    return data

In [18]:
%time df_modified = parallelize_dataframe(df, multiply_columns,num_cores)

CPU times: user 105 ms, sys: 139 ms, total: 244 ms
Wall time: 393 ms


In [19]:
df_modified.head()

,A,B,C,D,K
0,49,3,96,64,98
1,13,39,99,67,26
2,61,45,76,68,122
3,88,93,75,74,176
4,8,26,49,54,16


compare the above method of split/apply/combine to just "apply"

In [20]:
%time df['K'] = df['A'].apply(lambda x: x*2)

CPU times: user 33.4 ms, sys: 11.4 ms, total: 44.8 ms
Wall time: 45.5 ms


Applying the function to chunks took much more time! 

_Reason_: `multiprocessing` has to serialize the data, send the serialized data to each thread, then apply the transformation, then send the data back to main

# --> When would concurrency be useful?

Need to be able to amortize cost of concurrency

For your reference, here's how I typically structure my concurrency

In [ ]:
import multiprocessing
import time

In [ ]:
def serial_func(arg1):
    return arg1*2

In [ ]:
%time list(map(serial_func,[3,4,5]))

In [ ]:
if __name__ == '__main__':
    start_time=time.time()
    res_list=[]
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:   # start worker processes
        res_list = pool.map(serial_func,[3,4,5]) # see https://docs.python.org/3/library/multiprocessing.html

    print('main elapsed:',time.time()-start_time,'seconds')

In [ ]:
res_list